<center><font size=16>Final Project - HealthWatch</font></center>

# Necessary Libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
!pip install evaluate SpeechRecognition
!apt-get install portaudio19-dev python3-dev
!pip install pyaudio moviepy gradio mtcnn
!pip install python-dotenv
!pip install pymongo bcrypt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-

Not shared because it contains my DB password

In [5]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv('/content/drive/My Drive/Colab Notebooks/Final Project/.env')

# Access the MongoDB connection string
MongoDB = os.getenv("MongoDB")[9:126]

In [6]:
print(MongoDB[:26] + '<password #####>' + MongoDB[42:])

mongodb+srv://carl-ghanem:<password #####>@healthapp.zhnw7.mongodb.net/?retryWrites=true&w=majority&appName=HealthApp


# NLP

In [7]:
import pandas as pd
import numpy as np

import torch
import re
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, Trainer, pipeline, TrainingArguments
import evaluate
import os

## Code

In [8]:
mod_path = '/content/drive/MyDrive/Colab Notebooks/Final Project/NLP/Work/trainer_output/checkpoint-752'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
model0 = BertForSequenceClassification.from_pretrained(mod_path)

# Ensure model is in evaluation mode
model0.eval()

def predict_maladies(description, top_k=3):
    # Tokenize the input description
    inputs = tokenizer(description, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Get the logits from the model
    with torch.no_grad():
        outputs = model0(**inputs)
        logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.softmax(logits, dim=-1)

    # Get the top k maladies and their probabilities
    top_k_probs, top_k_indices = torch.topk(probabilities, top_k, dim=-1)

    # Convert indices to maladies (you'll need to define a list of malady labels corresponding to your model's output classes)
    maladies = [
    'CANCER','STROKE','DIABETES','FLU','ALLERGY','PNEUMONIA','DEMENTIA','COLD',
    'ARTHRITIS','EPILEPSY','ASTHMA','HEPATITIS','HERPES','AIDS','ANXIETY','MIGRAINE','HYPERTENSION','Unknown Disease'
    ]

    predicted_maladies = [maladies[idx] for idx in top_k_indices[0]]

    # Prepare the results
    results = [(predicted_maladies[i], top_k_probs[0][i].item()) for i in range(top_k)]

    return results


In [11]:
from collections import defaultdict
import json

# Extended list of known symptoms with some common variations
with open('/content/drive/My Drive/Colab Notebooks/Final Project/NLP/known_symptoms.json', "r") as file:
  known_symptoms = json.load(file)

# Function to clean and standardize text
def cleantxt(text):
    text = text.replace('-', ' ').replace('.', ' ')
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text.lower()

# Function to map text to known symptoms, considering variations
def map_to_symptoms(words):
    symptoms_found = set()
    for word in words:
        for symptom, variations in known_symptoms.items():
            for variation in variations:
                if variation in word:
                    symptoms_found.add(symptom)
    return symptoms_found

# Function to extract symptoms from patient input
def extract_symptoms(text):
    doc = cleantxt(text)  # Clean the input text
    words = doc.split()  # Split the cleaned text into words
    extracted_symptoms = set()

    # Check for phrases and individual words
    for i in range(len(words)):
        for j in range(i+1, len(words)+1):
            phrase = " ".join(words[i:j])
            symptoms_found = map_to_symptoms([phrase])
            if symptoms_found:
                extracted_symptoms.update(symptoms_found)

    return list(extracted_symptoms)

In [12]:
def Probabilities(description):
  if extract_symptoms(description) == []:
    return "Nonesense"

  dicti = {}
  top_maladies = predict_maladies(description)
  for malady, probability in top_maladies:
      dicti[malady] = probability
  return dicti

In [13]:
Text_sickness = lambda text : list(Probabilities(text).keys())[0]

## Test

In [14]:
Probabilities("I have red skin with rash and itchiness")

{'ALLERGY': 0.7546584010124207,
 'FLU': 0.21411943435668945,
 'AIDS': 0.019826717674732208}

In [15]:
Text_sickness("I have red skin with rash and itchiness")

'ALLERGY'

# Speech to text

In [16]:
import speech_recognition as sr

def speech_to_text(audio_file):
  recognizer = sr.Recognizer()

  try:
      with sr.AudioFile(audio_file) as source:
          audio = recognizer.record(source)
  except TypeError as e:
      print('Error in file type: ', e)

  try:
      text = recognizer.recognize_google(audio)
      return (text)
  except sr.UnknownValueError:
      return ("UNKNOWN")
  except sr.RequestError as e:
      raise ("Error with the service; {0}".format(e))


In [17]:
import moviepy.editor as mp
import tempfile

def split_audio_video(input_file):
  """
  Splits an input video file into separate audio and video files using temporary files.

  Args:
      input_file (str): Path to the input video file.

  Returns:
      tuple: A tuple containing the paths to the temporary audio and video files.
  """

  # Create temporary files with unique names using tempfile.NamedTemporaryFile
  with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio_file:
    audio_file_path = temp_audio_file.name

  # Load the video clip
    clip = mp.VideoFileClip(input_file)

    # Extract audio
    audioclip = clip.audio
    audioclip.write_audiofile(audio_file_path)

    return audio_file_path

In [18]:
def get_text(vid_loc):
  audio_file_path = split_audio_video(vid_loc)
  if audio_file_path is None:
      return 'UNKNOWN'

  # Perform speech-to-text conversion
  r = speech_to_text(audio_file_path)

  os.remove(audio_file_path)

  return r

# CV

In [19]:
import cv2
from mtcnn import MTCNN
import time
import numpy as np

## Helper methods

In [20]:
def detect_redness(image, landmarks, threshold=150):
    """
    Detects redness in specified regions of the face (around the eyes and nose).

    Args:
        image: The input image frame.
        landmarks: Facial landmarks detected (from MTCNN or dlib).
        threshold: Threshold for considering a pixel as red.

    Returns:
        redness_detected: Boolean indicating if redness is detected.
    """
    # Extract coordinates of the regions to check for redness
    left_eye = landmarks['left_eye']
    right_eye = landmarks['right_eye']
    nose = landmarks['nose']

    # Define regions around the eyes and nose
    eye_region_size = 20  # Adjust this based on the face size
    nose_region_size = 20

    # Create masks for the regions
    mask = np.zeros_like(image[:, :, 0])

    cv2.circle(mask, left_eye, eye_region_size, 255, -1)
    cv2.circle(mask, right_eye, eye_region_size, 255, -1)
    cv2.circle(mask, nose, nose_region_size, 255, -1)

    # Convert the image to HSV color space for better color analysis
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define red color range in HSV
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])

    # Create a red mask
    red_mask = cv2.inRange(hsv_image, lower_red, upper_red)
    red_mask = cv2.bitwise_and(red_mask, red_mask, mask=mask)

    # Count the number of red pixels
    red_pixels = cv2.countNonZero(red_mask)

    # If a significant number of pixels are red, consider it as redness detected
    return red_pixels > threshold


In [21]:
def analyze_lips_histogram(gray_mouth):
    """
    Analyzes the histogram of the mouth region to detect dryness.

    Args:
        gray_mouth: Grayscale image of the mouth region.

    Returns:
        dryness_detected: Boolean indicating if dryness is detected.
    """
    # Calculate the histogram
    hist = cv2.calcHist([gray_mouth], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()

    # Check for unusual peaks (typically around the mid to high brightness range)
    peak_threshold = 0.1  # This may need adjustment
    peaks = hist[150:255]  # Focus on the higher brightness values

    if np.max(peaks) > peak_threshold:
        return True
    return False


In [22]:
def detect_dry_lips(image, landmarks, edge_threshold=100):
    """
    Detects dryness on the lips by analyzing the texture (using Laplacian edge detection).

    Args:
        image: The input image frame.
        landmarks: Facial landmarks detected (from MTCNN or dlib).
        edge_threshold: Threshold for detecting edges (higher value = fewer edges).

    Returns:
        dry_lips_detected: Boolean indicating if dry lips are detected.
    """
    mouth_left = landmarks['mouth_left']
    mouth_right = landmarks['mouth_right']

    # Ensure coordinates are within image bounds
    x1, y1 = max(mouth_left[0], 0), max(mouth_left[1], 0)
    x2, y2 = min(mouth_right[0], image.shape[1]), min(mouth_right[1], image.shape[0])

    # Define the mouth region based on the landmarks
    if x2 > x1 and y2 > y1:  # Ensure valid region
        mouth_region = image[y1:y2, x1:x2]
    else:
        return False  # Invalid region, skip detection

    # Convert the mouth region to grayscale
    gray_mouth = cv2.cvtColor(mouth_region, cv2.COLOR_BGR2GRAY)

    # Apply Laplacian to detect edges/texture
    laplacian = cv2.Laplacian(gray_mouth, cv2.CV_64F)
    laplacian_abs = np.absolute(laplacian)
    laplacian_8u = np.uint8(laplacian_abs)

    # Count the number of edge pixels
    edge_pixels = cv2.countNonZero(laplacian_8u)

    # Thresholding to detect dryness
    if edge_pixels > edge_threshold: #or analyze_lips_histogram(gray_mouth):  # Adjust threshold based on testing - 1000
        return True
    return False


In [23]:
def detect_swelling(image, landmarks, eye_region_size=20, intensity_threshold=50, texture_threshold=20):
    """
    Detects swelling under the eyes by analyzing the region below the eyes.

    Args:
        image: The input image frame.
        landmarks: Facial landmarks detected (from MTCNN or dlib).
        eye_region_size: The size of the region below the eyes to analyze.
        intensity_threshold: Threshold for considering increased intensity as swelling.
        texture_threshold: Threshold for considering texture changes as swelling.

    Returns:
        swelling_detected: Boolean indicating if swelling is detected.
    """
    left_eye = landmarks['left_eye']
    right_eye = landmarks['right_eye']

    # Define the regions under the eyes
    left_eye_region = image[left_eye[1]:left_eye[1]+eye_region_size, left_eye[0]-eye_region_size//2:left_eye[0]+eye_region_size//2]
    right_eye_region = image[right_eye[1]:right_eye[1]+eye_region_size, right_eye[0]-eye_region_size//2:right_eye[0]+eye_region_size//2]

    def analyze_region(region):
        # Convert the region to grayscale
        gray_region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)

        # Apply GaussianBlur to smooth the image and focus on larger features
        blurred_region = cv2.GaussianBlur(gray_region, (5, 5), 0)

        # Calculate the mean intensity and texture variation
        mean_intensity = np.mean(blurred_region)
        texture_variation = np.std(blurred_region)  # Standard deviation as texture variation metric

        # Detect swelling based on intensity and texture
        if mean_intensity > intensity_threshold and texture_variation < texture_threshold:
            return True
        return False

    # Analyze both eye regions
    left_swelling = analyze_region(left_eye_region)
    right_swelling = analyze_region(right_eye_region)

    return left_swelling or right_swelling


In [24]:
def detect_pale_skin(image, landmarks, region_size=50, saturation_threshold=50, lightness_threshold=200):
    """
    Detects pale skin by analyzing the color saturation and lightness in specific regions of the face.

    Args:
        image: The input image frame.
        landmarks: Facial landmarks detected (from MTCNN or dlib).
        region_size: The size of the regions to analyze (forehead, cheeks).
        saturation_threshold: Threshold for considering the skin as pale based on saturation.
        lightness_threshold: Threshold for considering the skin as pale based on lightness.

    Returns:
        pale_skin_detected: Boolean indicating if pale skin is detected.
    """
    # Example regions to check (forehead, cheeks)
    forehead = (landmarks['left_eye'][0], landmarks['left_eye'][1] - region_size)
    left_cheek = (landmarks['mouth_left'][0], landmarks['mouth_left'][1])
    right_cheek = (landmarks['mouth_right'][0], landmarks['mouth_right'][1])

    regions = [forehead, left_cheek, right_cheek]
    pale_skin_detected = False

    for region in regions:
        x, y = region
        x1, y1 = max(0, x - region_size // 2), max(0, y - region_size // 2)
        x2, y2 = min(image.shape[1], x + region_size // 2), min(image.shape[0], y + region_size // 2)

        # Extract the region of interest
        roi = image[y1:y2, x1:x2]

        # Convert the ROI to the HSV color space
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

        # Calculate the mean saturation and lightness
        mean_saturation = np.mean(hsv_roi[:, :, 1])
        mean_lightness = np.mean(hsv_roi[:, :, 2])

        # If the region has low saturation and high lightness, consider it pale
        if mean_saturation < saturation_threshold and mean_lightness > lightness_threshold:
            pale_skin_detected = True
            break  # Exit loop early if paleness is detected in any region

    return pale_skin_detected


In [25]:
def detect_blackness(image, landmarks, threshold=150, lower_black_threshold=30):
    """
    Detects dark spots under the eyes by analyzing shades of black.

    Args:
        image: The input image frame.
        landmarks: Facial landmarks detected (from MTCNN or dlib).
        threshold: Threshold for considering the skin as dark.
        lower_black_threshold: Threshold for setting minimum detectable black.

    Returns:
         Boolean indicating if dark skin under the eyes is detected.
    """
    left_eye = landmarks['left_eye']
    right_eye = landmarks['right_eye']

    eye_region_size = 15  # Adjusted size for balance
    mask = np.zeros_like(image[:, :, 0])
    cv2.circle(mask, left_eye, eye_region_size, 255, -1)
    cv2.circle(mask, right_eye, eye_region_size, 255, -1)

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    lower_black = np.array([0, 0, lower_black_threshold])
    upper_black = np.array([180, 255, 80])

    blk_mask = cv2.inRange(hsv_image, lower_black, upper_black)
    blk_mask = cv2.bitwise_and(blk_mask, blk_mask, mask=mask)

    blk_pixels = cv2.countNonZero(blk_mask)

    eye_gray_values = gray_image[mask == 255]
    histogram, _ = np.histogram(eye_gray_values, bins=256, range=(0, 256))

    dark_pixels = histogram[0:50].sum()  # Pixels in the darkest range

    return blk_pixels > threshold or dark_pixels > (threshold // 10)  # Dark pixel count check

## Main methods

In [26]:
import numpy as np
import random
import tensorflow as tf
import os

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
cv2.setRNGSeed(42)


def clear_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)


def analyze_video(video_path):
    # extract_frames

    # Check if the file exists
    if not os.path.isfile(video_path):
        raise('Problem with path')

    # Check if the file has a common video extension
    valid_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.webm')
    if not video_path.lower().endswith(valid_extensions):
        raise('Problem with path')


    # Define the directory for storing frames
    frames_dir = '/content/frames/'

    # Clear the directory if it exists
    if os.path.exists(frames_dir):
        clear_directory(frames_dir)
    else:
        os.makedirs(frames_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % 6 == 0:  # Save every 6th frame
            cv2.imwrite(f"{frames_dir}frame_{frame_idx}.png", frame)

        frame_idx += 1

    cap.release()

    return analyze_images(frames_dir)



In [27]:
def analyze_images(frame_folder):
    sick_score = {
        'redness': 0,
        'dry_lips': 0,
        'pale_skin': 0,
        'swelling': 0,
        'black_eyes': 0
    }

    detector = MTCNN()

    for filename in sorted(os.listdir(frame_folder)):
        print('Processing ', filename)
        frame = cv2.imread(os.path.join(frame_folder, filename))


        # Detect faces in the frame
        faces = detector.detect_faces(frame)

        # Extract the bounding box and keypoints for the closest face detected
        face = max(faces, key=lambda face: face['box'][2] * face['box'][3])

        bounding_box = face['box']
        keypoints = face['keypoints']


        # Detecting sickness
        # Detect redness
        if detect_redness(frame, keypoints, 1500):
            print("Redness detected!")
            cv2.putText(frame, f'Redness Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2) # BGR
            sick_score['redness'] += 1

        # Detect dry lips
        if detect_dry_lips(frame, keypoints, edge_threshold=450):
            print("Dry lips detected!")
            cv2.putText(frame, f'Dry Lips Detected', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            sick_score['dry_lips'] += 1

        # Detect swelling under eyes
        if detect_swelling(frame, keypoints, intensity_threshold=105):
            print("Swelling under eyes detected!")
            cv2.putText(frame, f'Swelling Detected', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
            sick_score['swelling'] += 1

        # Detect pale skin
        if detect_pale_skin(frame, keypoints):
            print("Pale skin detected!")
            cv2.putText(frame, f'Pale Skin Detected', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
            sick_score['pale_skin'] += 1

        # Detect Black eyes - tiredness
        if detect_blackness(frame, keypoints, threshold=1200, lower_black_threshold=160):
            print("Black eyes detected!")
            cv2.putText(frame, f' Black Eyes Detected', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
            sick_score['black_eyes'] += 1

        # Draw bounding box
        cv2.rectangle(frame,
                        (bounding_box[0], bounding_box[1]),
                        (bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]),
                        (0, 155, 255), 2)

        # Draw facial landmarks
        cv2.circle(frame, keypoints['left_eye'], 2, (0, 155, 255), 2)
        cv2.circle(frame, keypoints['right_eye'], 2, (0, 155, 255), 2)
        cv2.circle(frame, keypoints['nose'], 2, (0, 155, 255), 2)
        cv2.circle(frame, keypoints['mouth_left'], 2, (0, 155, 255), 2)
        cv2.circle(frame, keypoints['mouth_right'], 2, (0, 155, 255), 2)

    return sick_score

In [28]:
def det_frame_count(vid_path):
    cap = cv2.VideoCapture(vid_path)

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    frames_in_20_seconds = int(frame_rate * 20)


    x = min(frames_in_20_seconds, frame_count)

    cap.release()
    return x


In [29]:
def cv_calculator(tot_frames, sick_scores):
    tot_frames = tot_frames // 20 # Because most of the frames are not used for detection and 20 is chosen because we capped it with 20s length
    weights = [0.3, 0.05, 0.3, 0.3, 0.05]

    cv_sc = weights[0] * sick_scores['redness'] + weights[1] * sick_scores['dry_lips'] + weights[2] * sick_scores['pale_skin'] + weights[3] * sick_scores['swelling'] + weights[4] * sick_scores['black_eyes']

    cv_sc = cv_sc / tot_frames

    return min(max(cv_sc/1.584, 0), 1)

# Random choice

In [30]:
import pandas as pd

In [31]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Final Project/GPT2/Augmented_Health_Dataset.xlsx')

In [32]:
df

,Malady,Response
0,cancer mild,"Based on the video and your description, your ..."
1,cancer mild,"Based on your video and symptoms described, it..."
2,cancer mild,Considering the video and the symptoms you've ...
3,cancer mild,"Your video and symptoms indicate that, your sy..."
4,cancer mild,The video analysis and your description sugges...
...,...,...
319,unknown severe,"Based on your video and symptoms described, it..."
320,unknown severe,Considering the video and the symptoms you've ...
321,unknown severe,"Your video and symptoms indicate that, I haven..."
322,unknown severe,The video analysis and your description sugges...


In [33]:
df['Disease'] = df['Malady'].apply(lambda x : x.split(" ", 1)[0])
df['severity'] = df['Malady'].apply(lambda x : x.split(" ", 1)[1])

dic5 = {
    'mild': 'healthy',
    'medium': 'sick',
    'severe': 'quite sick'
}

df['severity'] = df['severity'].apply(lambda x : dic5[x])

df.rename(columns = {'severity': 'Status'}, inplace=True)
df.rename(columns = {'Response': 'response'}, inplace=True)

In [34]:
df

,Malady,response,Disease,Status
0,cancer mild,"Based on the video and your description, your ...",cancer,healthy
1,cancer mild,"Based on your video and symptoms described, it...",cancer,healthy
2,cancer mild,Considering the video and the symptoms you've ...,cancer,healthy
3,cancer mild,"Your video and symptoms indicate that, your sy...",cancer,healthy
4,cancer mild,The video analysis and your description sugges...,cancer,healthy
...,...,...,...,...
319,unknown severe,"Based on your video and symptoms described, it...",unknown,quite sick
320,unknown severe,Considering the video and the symptoms you've ...,unknown,quite sick
321,unknown severe,"Your video and symptoms indicate that, I haven...",unknown,quite sick
322,unknown severe,The video analysis and your description sugges...,unknown,quite sick


In [35]:
import random as rnd

def get_resp(disease, cv):
  stats = {
      0: 'healthy',
      1: 'sick',
      2: 'quite sick'
  }

  ddf = df[(df['Disease'] == disease) & (df['Status'] == stats[2 if cv >= 0.7 else 1 if cv >= 0.3 else 0])]
  k = rnd.randint(0, len(ddf) - 1)
  return ddf.reset_index(drop=True).loc[k, 'response']

# Mongo DB

In [36]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = MongoDB

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print('Error: ', e)

Pinged your deployment. You successfully connected to MongoDB!


In [37]:
import bcrypt
import gradio as gr
import json

In [38]:
db = client["users_db"]
users_collection = db["users_collection"]

import gradio as gr

def hash_password(password):
    '''Encrypt the input password'''
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

def verify_password(stored_password, provided_password):
    '''Authenticate login'''
    return bcrypt.checkpw(provided_password.encode('utf-8'), stored_password)

def register_user(username, password, Name, Age, Weight, Height):
    '''Register new user to DB'''
    if users_collection.find_one({"username": username}):
        return "Username already exists!", False

    if username == '' or password == '' or Name == '' or Age == '' or Weight == '' or Height == '':
        return 'Please fill all the information.', False

    hashed_password = hash_password(password)
    users_collection.insert_one({"username": username, "password": hashed_password, "Name" : Name, "Age":Age, "Weight":Weight, "Height":Height})
    return "User registered successfully!", True


def login_user(username, password):
    '''Handle user login request'''
    user = users_collection.find_one({"username": username})
    if user is None:
        return "Invalid username or password!", False
    if verify_password(user["password"], password):
        return f"<font size = 5>Welcome back {username}!</font>", True
    else:
        return "Invalid username or password!", False


# Visibility GUI methods
def toggle_dropdown_0(visible):
    new_visibility = not visible
    return gr.update(visible=new_visibility), gr.update(visible=not new_visibility), new_visibility

def toggle_dropdown(visible):
    new_visibility = not visible
    return gr.update(visible=new_visibility), gr.update(visible=not new_visibility), gr.update(visible=new_visibility), gr.update(visible=not new_visibility), new_visibility, gr.update(visible=False)

In [39]:
from PIL import Image
image = Image.open("/content/drive/My Drive/Colab Notebooks/Final Project/Images/health.jpg")

In [40]:
import io
import base64

# Convert the image to base64
buffered = io.BytesIO() # Use io.BytesIO instead of BytesIO
image.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue()).decode()

# Create a Markdown string with the embedded image
img_tag = f"![Logo](data:image/jpeg;base64,{img_str})"

# GUI

In [41]:
from pathlib import Path
import base64
import re

In [42]:
def cleantxt(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

def dic_to_txt(d):
  s = ''
  sm = 0
  for i in d.items():
    sm += i[1]
    s += i[0] + ':\t\t' + str(i[1]*100)[:6] + '%\t\t\t'
    if sm > 0.9:
      break

  return s + '\n\n'

def dic_to_keys(d):
    s = ''
    d = {'redness': 'redness on your face', 'dry_lips': 'dry lips', 'pale_skin': 'pale skin', 'swelling': 'swelling under your eyes', 'black_eyes': 'dark spots under your eyes'}
    for i in d.keys():
        processed_word = d[str(i)]
        s += processed_word + ', '
    s = s[:-2] + '.'  # Remove the last comma and space, and add a period.
    s = s[::-1].replace(' ,', ' dna ', 1)[::-1]  # Replace the last comma with "and".
    return s

In [43]:
def gen_txt(score, sic, dic, analysis_result):
  context = {
        0 : 'look well',
        1 : 'look mildly ill or possibly just tired',
        2 : 'look badly ill'
    }

  medical_conditions = {
    "cold": "a",
    "arthritis": "",
    "epilepsy": "",
    "asthma": "",
    "hepatitis": "",
    "herpes": "",
    "aids": "",
    "anxiety": "",
    "migraine": "a",
    "hypertension": "",
    "allergy" : "an"
  }

  # Make the condition name bold and underlined
  t1 = f"Your video scored {str(score)[:4]} which means you {context[int(score > 0.3) + int(score > 0.7)]}"
  t2 = f'. Our model says you most probably have <b>{medical_conditions[sic.lower()]} {sic}</b>'

  # Underline the detected characteristics
  t = f'\n\n<b>Detected video characteristics:</b>\t{dic_to_keys(analysis_result)}'

  t3 = f'\n\nIn fact, check out the results (% match):\n{dic_to_txt(dic)}'

  t4 = get_resp(sic.lower(), score)


  return t1 + t2 + t + t3 + t4

In [44]:
import tempfile
import traceback
def generate_response(message, video):

    with open('/content/drive/My Drive/Colab Notebooks/Final Project/NLP/unknown.json', "r") as file:
      unknown_prompts = json.load(file)

    predef = unknown_prompts[str(rnd.randint(0, 6))]

    # Create a log file to write debug information
    debug_log_path = "/content/debug_log.txt"

    with open(debug_log_path, "w") as log_file:
        try:
            # Write the type and initial content of video to the log
            log_file.write(f"Video input type: {type(video)}\n")
            log_file.write(f"Video input content: {video}\n")

            # Analyze text
            log_file.write(f"Text input type: {type(message)}\n")
            log_file.write(f"Text input content: {message}\n")

            if message == None:
              log_file.write(f"\nMessage: NO MSG\n")

            log_file.write(f"\nMessage is being processed: {message}\n")


            dic = Probabilities(message)

            if dic == "Nonesense":
              log_file.write(f"\nMessage is nonsense\n")
              return predef

            sic = Text_sickness(message)

            # Analyze video
            analysis_result = analyze_video(video)
            frtmp = det_frame_count(video)
            score = cv_calculator(frtmp, analysis_result)

            log_file.write(f"\nAnalysis result: {analysis_result}\n")
            log_file.write(f"\nScore: {score}\n")

            # Get response
            st = gen_txt(score, sic, dic, analysis_result)
            log_file.write(f"Generated text: {st}\n")

        except Exception as e:
            log_file.write(f"Error: {e}\n")
            log_file.write(f"Traceback: {traceback.format_exc()}\n")
            print(e)
            st = 'Error: ' + e

    return st

In [45]:
def chatbot_response(message, pick, video, visible_state):
    corr = {
        "1st choice": "video0.mp4",
        "2nd choice": "video1.mp4",
        "3rd choice": "video5.mp4"
    }

    if visible_state:
      # If no video is uploaded, show an error message
      if not pick in list(corr.keys()):
        return None, message, gr.update(value=None), gr.update(value="<span style='color:red;'>Please attach a video file.</span>", visible=True)
      else:
        video = '/content/drive/My Drive/Colab Notebooks/Final Project/CV/Videos/' + corr[pick]
    else:
      if video is None:
        return None, message, gr.update(value=None), gr.update(value="<span style='color:red;'>Please attach a video file.</span>", visible=True)
    bub2 = False
    if str(message).strip() == '':
      message = get_text(video)
      bub2 = True

    # Simulate a chatbot response
    response = generate_response(message, video)

    # Convert video to a base64 string for embedding
    video_data = Path(video).read_bytes()
    video_base64 = base64.b64encode(video_data).decode('utf-8')

    # Create the video bubble
    video_bubble =  f'<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">' \
                    f'<div style="margin-bottom: 5px; width: 250px; height: 150px;">' \
                    f'<video controls style="border-radius: 15px; width: 100%; height: 100%; object-fit: cover;">' \
                    f'<source src="data:video/mp4;base64,{video_base64}" type="video/mp4">' \
                    f'Your browser does not support the video tag.' \
                    f'</video>' \
                    f'</div>' \
                    f'</div>'

    # Create the user's message bubble
    if not bub2:
      user_bubble = f'<div class="chat-bubble user-bubble" style="text-align: left; margin-bottom: 10px;">' \
                  f'{message}' \
                  f'</div>'
    else:
      user_bubble = f'<div class="chat-bubble user-bubble" style="text-align: right; margin-bottom: 10px;">' \
                  f'You said: {message}' \
                  f'</div>'

    # Create the bot's response bubble
    bot_bubble = f'<div class="chat-bubble bot-bubble" style="text-align: left; margin-bottom: 10px;">' \
                f'{response}' \
                f'</div>'

    # Return the combined chat log and reset video input
    if message:
        return gr.update(value=video_bubble + user_bubble + bot_bubble, visible=True), "", gr.update(value=None), gr.update(visible=False)
    return gr.update(value=video_bubble + user_bubble + bot_bubble, visible=True), "", gr.update(value=None), gr.update(visible=False)

In [46]:
# Creating the Gradio interface
with gr.Blocks() as demo:
    with gr.Column():

        # Conditional Login Page Title
        login_title = gr.Markdown("<center><b><font size=16> Login Page </font></b></center><br><br>", visible=True)

        with gr.Row():
            # Stacked username and password on the left
            with gr.Column(scale=1):
                with gr.Row():
                    Name = gr.Textbox(label="Full name", visible=False)
                    username = gr.Textbox(label="Username")
                    password_reg = gr.Textbox(label="Password", type="password", visible=False)

                with gr.Row():
                    pass

                with gr.Row():
                    password = gr.Textbox(label="Password", type="password")
                    Age = gr.Textbox(label="Age", visible=False)
                    Weight = gr.Textbox(label="Weight in KG", visible=False)
                    Height = gr.Textbox(label="Height in cm", visible=False)

                with gr.Row():
                    pass
                with gr.Row():
                    pass
                with gr.Row():
                    pass
                with gr.Row():
                    pass
                with gr.Row():
                    login_button = gr.Button("Login", visible=True)
                    register_button = gr.Button("Register", visible=False)

                with gr.Row():
                    reg = gr.Button("Don't have an account? Register...", visible=True)
                    log = gr.Button("Already have an account? Login...", visible=False)


            # Centered image on the right
            with gr.Column(scale=1):
                #image_component = gr.Image(value=image, visible=True)
                img_test_tag = f"<center> <img src=data:image/jpeg;base64,{img_str} style=\"max-width: 100%; height: auto;\"></center>"
                image_component = gr.Markdown(value=img_test_tag, visible=True, elem_id = "imgl")

        with gr.Row():
            inner_title = gr.Markdown("<center><b><font size=16> HealthWatch </font></b></center>", visible=False)
        with gr.Row():
            pass

        with gr.Row():
            with gr.Column(scale=1):
                login_output = gr.Markdown(visible=True)
            with gr.Column(scale=1):
                pass
            with gr.Column(scale=1):
                logout = gr.Button("Logout", visible=False)


        ########################################################
        chat_log = gr.HTML("", elem_id="chat-log", visible=False)
        with gr.Row():
            toggle_btn = gr.Button("Show/Hide Dropdown", visible=False)
        with gr.Row():
            txt = gr.Textbox(placeholder="Type your message here...", lines=10, elem_id="text-upload", visible=False)
            video = gr.Video(label="Attach a video", elem_id="video-upload", visible=False)
            pick = gr.Dropdown(["N/A", "1st choice", "2nd choice", "3rd choice"], label="Test Video", info="Pick a video for testing", visible=False, elem_id = "pick-upload")

        with gr.Row():
            btn = gr.Button("Send", visible=False)

        with gr.Row():
            error_msg = gr.HTML(visible=False)
        with gr.Row():
            ai = gr.Markdown(value = "<center>HealthWatch can make mistakes. If very ill, contact a healthcare professional.<center>",visible=False)
        with gr.Row():
            privacy_policy = gr.Markdown(value = "<a href = 'https://drive.google.com/file/d/1Tu8hqcZw_RyyitdwCd6vx7E9qyApl5Ms/view?usp=sharing'>Privacy Policy</a>",visible=True)


        visible_state = gr.State(value=False)
        visible_state2 = gr.State(value=True)

        # Initial State
        is_logged_in = gr.State(value=False)

        # Link the button to the chatbot response function
        btn.click(chatbot_response, inputs=[txt, pick, video, visible_state], outputs=[chat_log, txt, video, error_msg])

        # Link the toggle button to the dropdown visibility function
        toggle_btn.click(toggle_dropdown_0, inputs=visible_state, outputs=[pick, video, visible_state])

        #######################################################

        # Authentication Logic
        def handle_login(username, password):
            message, success = login_user(username, password)
            if success:
                return (
                    message,                              # login_output
                    gr.update(visible=False),             # login_title
                    gr.update(visible=True),              # Inner title
                    gr.update(visible=True),              # chat_log
                    gr.update(visible=True),              # toggle_btn
                    gr.update(visible=True),              # txt
                    gr.update(visible=True),              # video
                    gr.update(visible=True),              # btn
                    gr.update(visible=False),             # username
                    gr.update(visible=False),             # password
                    gr.update(visible=False),             # login_button
                    gr.update(visible=False),             # register_button
                    gr.update(visible=False),             # reg
                    gr.update(visible=False),             # log
                    gr.update(visible=False),             # Image
                    gr.update(visible=True),              # logout
                    gr.update(value = "", visible=True),  # Error msg
                    gr.update(visible=False),             # Pick
                    gr.update(visible=False),             # Visible state
                    gr.update(visible=True),              # AI disclaimer
                )
            else:
                return (
                    message,
                    gr.update(visible=True),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=True),
                    gr.update(visible=True),
                    gr.update(visible=bool(visible_state2)),
                    gr.update(visible=not bool(visible_state2)),
                    gr.update(visible=bool(visible_state2)),
                    gr.update(visible=not bool(visible_state2)),
                    gr.update(visible=True),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                    gr.update(visible=False),
                )

        def handle_logout():
            return (
                gr.update(visible=True),                 # login_title
                gr.update(visible=False),                # Inner title
                gr.update(value = "", visible=False),    # chat_log
                gr.update(visible=False),                # toggle_btn
                gr.update(value = "", visible=False),    # txt
                gr.update(visible=False),                # video
                gr.update(visible=False),                # btn
                gr.update(value = "", visible=True),     # username
                gr.update(value = "", visible=True),     # password
                gr.update(visible=True),                 # login_button
                gr.update(visible=False),                # register_button
                gr.update(visible=True),                 # reg
                gr.update(visible=False),                # log
                gr.update(visible=True),                 # Image
                gr.update(visible=False),                # Logout
                gr.update(value = "N/A", visible=False), # Pick
                gr.update(visible=False),                # Inner title
                gr.update(visible=False),                # Error msg
                gr.update(value=""),                     # Login output
                gr.update(visible=False),                # AI disclaimer
            )


        def handle_register(username, password, Name, Age, Weight, Height):
            message, success = register_user(username, password, Name, Age, Weight, Height)

            if success:
                return (
                    message,
                    gr.update(visible=True),   # login_button
                    gr.update(visible=False),  # register_button
                    gr.update(visible=False),  # log
                    gr.update(visible=True),   # reg
                    gr.update(value = "", visible=False),
                    gr.update(value = "", visible=False),
                    gr.update(value = "", visible=False),
                    gr.update(value = "", visible=False),
                    gr.update(value = "", visible=False),
                    gr.update(visible=True)
                )
            return (
                    message,
                    gr.update(visible=False),   # login_button
                    gr.update(visible=True),    # register_button
                    gr.update(visible=True),    # log
                    gr.update(visible=False),   # reg
                    gr.update(visible=True),
                    gr.update(visible=True),
                    gr.update(visible=True),
                    gr.update(visible=True),
                    gr.update(visible=True),
                    gr.update(visible=False)
                )

        # Toggle between login and register views
        def toggle_auth_view():
            return (
                gr.update(visible=False),  # login_button
                gr.update(visible=True),   # register_button
                gr.update(visible=True),   # log
                gr.update(visible=False),  # reg
                gr.update(value = "", visible=True),
                gr.update(value = "", visible=True),
                gr.update(value = "", visible=True),
                gr.update(value = "", visible=True),
                gr.update(value = "", visible=True),
                gr.update(visible=False)
            )

        def toggle_login_view():
            return (
                gr.update(visible=True),   # login_button
                gr.update(visible=False),  # register_button
                gr.update(visible=False),  # log
                gr.update(visible=True),   # reg
                gr.update(value = "", visible=False),
                gr.update(value = "", visible=False),
                gr.update(value = "", visible=False),
                gr.update(value = "", visible=False),
                gr.update(value = "", visible=False),
                gr.update(visible=True)
            )

        # Login/out
        login_button.click(handle_login, inputs=[username, password], outputs=[login_output, login_title, inner_title, chat_log, toggle_btn, txt, video, btn, username, password, login_button, register_button, reg, log, image_component, logout, error_msg, pick, visible_state, ai])
        logout.click(handle_logout, inputs=[], outputs=[login_title, inner_title, chat_log, toggle_btn, txt, video, btn, username, password, login_button, register_button, reg, log, image_component, logout, pick, inner_title, error_msg, login_output, ai])

        # Register
        register_button.click(handle_register, inputs=[username, password_reg, Name, Age, Weight, Height], outputs=[login_output, login_button, register_button, log, reg, password_reg, Name, Age, Weight, Height, password])

        # Switch view
        reg.click(toggle_auth_view, outputs=[login_button, register_button, log, reg, password_reg, Name, Age, Weight, Height, password])
        log.click(toggle_login_view, outputs=[login_button, register_button, log, reg, password_reg, Name, Age, Weight, Height, password])

In [47]:
demo.css = """

#chat-log {
    height: 250px;
    max-height: 400px;
    overflow: auto !important;
    border: 1px solid #e5e5ea;
    padding: 10px;
    padding: 10px;
    margin-bottom: 10px;
    background-color: white;
    border-radius: 15px;
    box-sizing: border-box;
    display: flex;
    flex-direction: column;
    justify-content: flex-start;
    scroll-behavior: smooth;
}

#chat-log::-webkit-scrollbar {
  width: 12px;
}

#chat-log::-webkit-scrollbar-track {
  background: #f1f1f1;
}

#chat-log::-webkit-scrollbar-thumb {
    background: #555;
    border-radius: 10px;
}

#chat-log::-webkit-scrollbar-thumb:hover {
    background: #333;
}

.chat-bubble {
    max-width: 60%;
    word-wrap: break-word;
    overflow-wrap: break-word;
    white-space: pre-wrap;
    padding: 8px 12px;
    border-radius: 15px;
    margin-bottom: 10px;
}

.user-bubble {
    background-color: #0084ff;
    color: white;
    border-radius: 15px 15px 0 15px;
    text-align: left;
    float: right;
    clear: both;
}

.bot-bubble {
    background-color: #e5e5ea;
    color: black;
    border-radius: 15px 15px 15px 0;
    margin-right: auto;
    text-align: left;
    clear: both;
    margin-top: 10px;
    float: left;
}

.bot-bubble b {
    font-weight: bold;
    color: inherit;
}

.hidden {
    display: none !important;
}


#video-upload, #text-upload, #pick-upload {
    height: 300px;
    margin-right: 10px;
}

textarea {
  resize: none;
}

#imgl {
    text-align: right;
}

"""

Eurisko user<br>
Username: Eurisko<br>
Password: 123456789

In [48]:
# Launch the interface
demo.launch(inbrowser=True)#, debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://afea2139a8884546ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
